In [2]:
import pandas as pd

### Game Analysis

In [38]:
df_game = pd.read_csv('E:\COM480-DataVis\datavis-project-2022-y3\Data\match_results.csv', 
                      parse_dates=['date'],
                      usecols=['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament'])
df_game.head()

,date,home_team,away_team,home_score,away_score,tournament
0,1872-11-30,Scotland,England,0,0,Friendly
1,1873-03-08,England,Scotland,4,2,Friendly
2,1874-03-07,Scotland,England,2,1,Friendly
3,1875-03-06,England,Scotland,2,2,Friendly
4,1876-03-04,Scotland,England,3,0,Friendly


First thing first, we are going to filter the dataframe by year of which matches were held, as well as tournament that are methods of qualifaction toward FIFA world cup, or just world cup games. To be specific, we only keep games took place after 1990 that were part of the following tournaments:
- AFC Asian Cup
- AFC Asian Cup qualification
- African Cup of Nations
- African Cup of Nations qualification
- Copa América
- Copa América qualification
- FIFA World Cup
- FIFA World Cup qualification
- Oceania Nations Cup
- UEFA Euro
- UEFA Euro qualification
- UEFA Nations League


In [39]:
# Filter data by year and tournament

# Only keep games from 1990
df_game = df_game[df_game.date.dt.year >= 1990]

# Only keep tournament that are methods of qualifaction for world cup, that is:
tournaments=['AFC Asian Cup', 'AFC Asian Cup qualification', 'African Cup of Nations', 'African Cup of Nations qualification', 'Copa América', 'Copa América qualification'
             'FIFA World Cup', 'FIFA World Cup qualification', 'Oceania Nations Cup', 'UEFA Euro', 'UEFA Euro qualification', 'UEFA Nations League']

df_game = df_game[df_game.tournament.isin(tournaments)]

In [40]:
df_game

,date,home_team,away_team,home_score,away_score,tournament
16061,1990-03-02,Algeria,Nigeria,5,1,African Cup of Nations
16062,1990-03-03,Cameroon,Zambia,0,1,African Cup of Nations
16063,1990-03-03,Ivory Coast,Egypt,3,1,African Cup of Nations
16064,1990-03-03,Senegal,Kenya,0,0,African Cup of Nations
16066,1990-03-05,Algeria,Ivory Coast,3,0,African Cup of Nations
...,...,...,...,...,...,...
43178,2022-01-30,Senegal,Equatorial Guinea,3,1,African Cup of Nations
43184,2022-02-02,Burkina Faso,Senegal,1,3,African Cup of Nations
43185,2022-02-03,Cameroon,Egypt,0,0,African Cup of Nations
43186,2022-02-05,Cameroon,Burkina Faso,3,3,African Cup of Nations


In [41]:
# Add a column to indicate the result of the matches:
# 0 - home team win
# 1 - draw
# 2 - away team win

df_game['result'] = df_game.apply(lambda Row: 0 if Row['home_score'] > Row['away_score'] 
                                                else (1 if Row['home_score'] < Row['away_score'] else 0), axis=1)

df_game.head()

,date,home_team,away_team,home_score,away_score,tournament,result
16061,1990-03-02,Algeria,Nigeria,5,1,African Cup of Nations,0
16062,1990-03-03,Cameroon,Zambia,0,1,African Cup of Nations,1
16063,1990-03-03,Ivory Coast,Egypt,3,1,African Cup of Nations,0
16064,1990-03-03,Senegal,Kenya,0,0,African Cup of Nations,0
16066,1990-03-05,Algeria,Ivory Coast,3,0,African Cup of Nations,0


In [35]:
df_game.groupby('home_team').agg({'result': 'sum'}).sort_values(by=['result'])

,result
home_team,
Yugoslavia,1
Bermuda,1
Somalia,1
Puerto Rico,2
Cook Islands,2
...,...
Andorra,57
Luxembourg,61
Faroe Islands,61


In [36]:
df_game.groupby('away_team').agg({'result': 'sum'}).sort_values(by=['result'])

,result
away_team,
Cayman Islands,0
Cook Islands,0
Turks and Caicos Islands,0
British Virgin Islands,0
Djibouti,0
...,...
Italy,53
Iran,53
Portugal,55


In [42]:
df_game.groupby('away_team').size()

away_team
Afghanistan       16
Albania           86
Algeria           98
American Samoa     4
Andorra           67
                  ..
Wales             84
Yemen             49
Yugoslavia         4
Zambia            90
Zimbabwe          70
Length: 213, dtype: int64

### Player Analysis
In this part, you will leverage Hive to perform exploratory analysis of data published by the [Open Data Platform Swiss Public Transport](https://opentransportdata.swiss).
Format: the dataset is originally presented as a collection of textfiles with fields separated by ';' (semi-colon). For efficiency, the textfiles have been converted into Optimized Row Columnar ([_ORC_](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+ORC)) file format. 
Location: you can find the data in ORC format on HDFS at the path `/data/sbb/part_orc/istdaten`.

The full description from opentransportdata.swiss can be found in <https://opentransportdata.swiss/de/cookbook/ist-daten/> in four languages. There may be inconsistencies or missing information between the translations.. In that case we suggest you rely on the German version and use an automated translator when necessary. We will clarify if there is still anything unclear in class and Slack. Here we remind you the relevant column descriptions:

- `BETRIEBSTAG`: date of the trip
- `FAHRT_BEZEICHNER`: identifies the trip
- `BETREIBER_ABK`, `BETREIBER_NAME`: operator (name will contain the full name, e.g. Schweizerische Bundesbahnen for SBB)
- `PRODUKT_ID`: type of transport, e.g. train, bus
- `LINIEN_ID`: for trains, this is the train number
- `LINIEN_TEXT`,`VERKEHRSMITTEL_TEXT`: for trains, the service type (IC, IR, RE, etc.)
- `ZUSATZFAHRT_TF`: boolean, true if this is an additional trip (not part of the regular schedule)
- `FAELLT_AUS_TF`: boolean, true if this trip failed (cancelled or not completed)
- `HALTESTELLEN_NAME`: name of the stop
- `ANKUNFTSZEIT`: arrival time at the stop according to schedule
- `AN_PROGNOSE`: actual arrival time
- `AN_PROGNOSE_STATUS`: show how the actual arrival time is calcluated
- `ABFAHRTSZEIT`: departure time at the stop according to schedule
- `AB_PROGNOSE`: actual departure time
- `AB_PROGNOSE_STATUS`: show how the actual departure time is calcluated
- `DURCHFAHRT_TF`: boolean, true if the transport does not stop there


In [8]:
df_player = pd.read_csv('E:\COM480-DataVis\datavis-project-2022-y3\Data\players_21_value.csv')
df_player.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
